In [14]:
# -*- coding: utf-8 -*-
# book_dim

import pandas as pd
import numpy as np
import re
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import date
from datetime import datetime
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import bigquery_storage
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
# Custom Functions
from gcp_getsecrets import get_gcp_secret
from gcp_postbucket import save_bucket
from gcp_getbucket import get_bucket_csv
from gcp_getbigquery import read_bq_table
from gcp_savebigquery import save_to_bq

#ARCHIVE PATH
archive_path = r'D:\\BleakRiverDataServ\\Clients\\Button\\RoyaltyCheck\\'
# Define project
project_id = f"button-datawarehouse"
# Define storage bucket for push
bucket_name = "cs-royalties-test"  # Replace with your bucket name
# Scopes for Service Account
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/admin.datatransfer",
    "https://www.googleapis.com/auth/bigquery",
    "https://www.googleapis.com/auth/cloud-platform"
]
# Define secrets to fetch
p_consumerkey = get_gcp_secret(project_id, "wc_consumer_key", "latest")
p_consumersecret = get_gcp_secret(project_id, "wc_consumer_secret", "latest")
secret_id_for_sa_key = "storage_sa_key" # The secret you just created
# get those secrets
sa_key_json_string = get_gcp_secret(project_id, secret_id_for_sa_key)
credentials_info = json.loads(sa_key_json_string)
credentials = service_account.Credentials.from_service_account_info(credentials_info, scopes = SCOPES)
storage_client = storage.Client(credentials=credentials, project=project_id)
bq_client = bigquery.Client(credentials=credentials)

""" 
0. BOOKS_INFO_SOURCE, BUNDLE_INFO_SOURCE
"""
# Big Query Book Master
books_info_database = "master_dim_data"
books_info_table = "books_info_source"
#booksinfo_df = read_bq_table(f"button-datawarehouse.master_dim_data.testtable", client=bq_client)
booksinfodf = read_bq_table(f"{project_id}.{books_info_database}.{books_info_table}", bq_client)

# Big Query Bundle Master
bundle_info_database = "master_dim_data"
bundle_info_table = "bundle_info_source"
#booksinfo_df = read_bq_table(f"button-datawarehouse.master_dim_data.testtable", client=bq_client)
bundleinfodf = read_bq_table(f"{project_id}.{bundle_info_database}.{bundle_info_table}", bq_client)

# Clean Books Info Source

booksinfodf['Audiobook_Royalty_Rate'] = booksinfodf['Audiobook_Royalty_Rate'].fillna('0.25')
booksinfodf['Print_Royalty_Rate'] = booksinfodf['Print_Royalty_Rate'].fillna('0.15')
booksinfodf['eBook_Royalty_Rate'] = booksinfodf['eBook_Royalty_Rate'].fillna('0.25')
booksinfodf['Print_ISBN'] = booksinfodf['Print_ISBN'].fillna('NA-Print-' + booksinfodf['Short_Hand_Title'])
booksinfodf['eBook_ISBN'] = booksinfodf['eBook_ISBN'].fillna('NA-eBook-' + booksinfodf['Short_Hand_Title'])
booksinfodf['Audiobook_ISBN'] = booksinfodf['Audiobook_ISBN'].fillna('NA-Audiobook-' + booksinfodf['Short_Hand_Title'])
booksinfodf['Hardcover_ISBN'] = booksinfodf['Hardcover_ISBN'].fillna('NA-Hardcover-' + booksinfodf['Short_Hand_Title'])
booksinfodf['Number_of_Pages'] = booksinfodf['Number_of_Pages'].fillna(0)

"""
1. START WITH SCB BOOKS AND WC BOOKS CLEAN
"""
# Set Dtype Dict

source_book_dtypes = { "SourceTitle" : object,
                      "CleanTitle" : object,
                      "BookType" :object,
                      "Source" :object
                      }

# Grab from Stage, set dtypes
scb_clean = get_bucket_csv(bucket_name, 'stage/scb_stage/SCB_Books_All.csv', dtype_spec=source_book_dtypes, client=storage_client).reset_index(drop=True).drop_duplicates()
wc_clean = get_bucket_csv(bucket_name, 'stage/woocom_stage/WooCom_Books_All.csv', dtype_spec=source_book_dtypes, client=storage_client).reset_index(drop=True).drop_duplicates()
# Merch & Bundle
wc_merch = get_bucket_csv(bucket_name, 'stage/woocom_stage/WooCom_Merch_All.csv', client=storage_client).reset_index(drop=True).drop_duplicates()
wc_bundle = get_bucket_csv(bucket_name, 'stage/woocom_stage/WooCom_Bundle_All.csv', client=storage_client).reset_index(drop=True).drop_duplicates()

"""
2. APPEND THE FRAMES TOGETHER
"""
wc_scb = pd.concat([wc_clean, scb_clean], ignore_index=True)
wc_scb['key'] = 0
booksinfodf['key'] = 0
cross_join = booksinfodf.merge(wc_scb, on='key', how='outer')
booksf = cross_join.copy()
booksf = booksf.set_index('Book_Title', drop=False)
booksf = booksf.rename(columns={'Book_Title': 'MasterTitle'})

# Initial cleaning step: strip leading and trailing spaces from Master and Clean
booksf['CleanTitle'] = booksf['CleanTitle'].apply(lambda x : x.strip())
booksf['MasterTitle'] = booksf['MasterTitle'].fillna('')
booksf['MasterTitle'] = booksf['MasterTitle'].apply(lambda x : x.strip())
booksf['TestTitle'] = booksf['CleanTitle']

# Word and string replacements
words_to_replace = r'\s+\b(the|and)\b\s+|\s*\|\s*'

# [():] matches any single character in the set
# The other words are matched directly
chars_and_phrases_to_remove = r"[():]|pre-order|pre order|paperback|hardcover|'|├│n|ΓÇ£|ΓÇ¥|out of print|digital only|[//:]|Γäó"
# --- MasterTitleL chained cleaning operation ---
booksf['MasterTitleL'] = (
    booksf['MasterTitle']
    .str.lower()
    .str.replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)
    .str.replace('digital only // out of print', '', regex=True)
    .str.replace(words_to_replace, ' ', regex=True)        # Replace ' the ', ' and ', '|' with a space
    .str.replace(chars_and_phrases_to_remove, '', regex=True) # Remove all other unwanted text
    .str.replace(r'\s+', ' ', regex=True)                  # Collapse multiple spaces into one
    .str.strip()                                           # Remove leading/trailing spaces
)

# --- TestTitleL chained cleaning operation ---
booksf['TestTitleL'] = (
    booksf['TestTitle']
    .str.lower()
    .str.replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)
    .str.replace('digital only // out of print', '', regex=True)
    .str.replace(words_to_replace, ' ', regex=True)        # Replace ' the ', ' and ', '|' with a space
    .str.replace(chars_and_phrases_to_remove, '', regex=True) # Remove all other unwanted text
    .str.replace(r'\s+', ' ', regex=True)                  # Collapse multiple spaces into one
    .str.strip()                                           # Remove leading/trailing spaces
)

# Specific Cleanups
# Define a dictionary for these direct string replacements
replacements = {
    'future limited edition': 'future limited edition hilborn',
    'helium limited edition': 'helium limited edition francisco',
    'madness vase': 'madness vase gibson'
}

# Use pd.Series.replace() with the dictionary
# regex=True is crucial to replace these substrings anywhere in the title
booksf['TestTitleL'] = booksf['TestTitleL'].replace(replacements, regex=True)

# 2. Use .loc for the conditional update
# This is much faster than an if/else lambda
condition = (booksf['TestTitleL'] == 'poetry by chance')
booksf.loc[condition, 'TestTitleL'] += ' an anthology of poems powered by metaphor dice'

# Strip one more time to catch any whitepaces added during cleanup
booksf['TestTitleL'] = booksf['TestTitleL'].apply(lambda x : x.strip())
booksf['MasterTitleL'] = booksf['MasterTitleL'].apply(lambda x : x.strip())

""" 
3. FUZZY MATCH MASTER TO TEST
"""
booksf2 = booksf.copy()

# 1. Prepare the vectorizer's vocabulary
all_titles = pd.concat([booksf2['MasterTitleL'], booksf2['TestTitleL']]).dropna().unique()
vectorizer = TfidfVectorizer().fit(all_titles)

# 2. Create TF-IDF vectors, filling NaNs
vectors1 = vectorizer.transform(booksf2['MasterTitleL'].fillna(''))
vectors2 = vectorizer.transform(booksf2['TestTitleL'].fillna(''))

# 3. Calculate similarity
# Normalize the row vectors to unit length
vectors1_normalized = normalize(vectors1)
vectors2_normalized = normalize(vectors2)

# Compute the dot product of corresponding row vectors
# This is equivalent to cosine similarity for normalized vectors
similarity_scores = np.asarray(vectors1_normalized.multiply(vectors2_normalized).sum(axis=1)).flatten()

# 4. Add the new score to booksf2
booksf2['TfidfSimilarity'] = similarity_scores

In [8]:
wc_clean.dtypes

SourceTitle    object
CleanTitle     object
BookType       object
Source         object
dtype: object

In [4]:
scb_clean.dtypes

SourceTitle    object
CleanTitle     object
BookType       object
Source         object
dtype: object

In [11]:
archive_path = r'D:\BleakRiverDataServ\Clients\Button\RoyaltyCheck'

In [ ]:
frat2 = booksf2['MasterTitle'] == "You Better Be Lightning"
bookfilt2 = booksf2[frat2].copy()
bookfilt2.to_csv(r'D:\BleakRiverDataServ\Clients\Button\Notes\bookmatch.csv')

In [15]:
booksf2.to_csv(archive_path + r'\bookmatch_full.csv')